In [ ]:
import json
import pandas as pd

year = 2019

allgames = pd.DataFrame()
allPlays = pd.DataFrame()

for year, month, day in [(year, month, day) for year in [year] for month in range(3,11) for day in range(1,30)]:
    filename = f'../data/schedule/{year}-{month:02d}-{day:02d}.json'
    with open(filename, 'r') as infile:
        data = json.loads(infile.read())
        games = data["dates"][0]["games"] if data["dates"] else []
        norm = pd.json_normalize(games)
        allgames = pd.concat([allgames, norm])
        
for index, row in allgames.iterrows():
    gamepk = row["gamePk"]
    filename = f'../data/games/{gamepk}.json'
    try:
        with open(filename, 'r') as infile:
            data = json.loads(infile.read())
            plays = data["allPlays"] if data["allPlays"] else []
            norm = pd.json_normalize(plays)
            allPlays = pd.concat([allgames, norm])
    except:
        pass
        

In [ ]:
allgames.groupby("seriesDescription").count()

In [ ]:
allgames[allgames['seriesDescription']=='Regular Season'].groupby("teams.home.team.name").count()

In [ ]:
import json
import pandas as pd
from jsonpath_ng import jsonpath, parse

filename = f'../data/games/564721.json'

with open(filename, 'r') as infile:
    #atbats = pd.json_normalize([rec["result"] | rec["count"] | rec["about"] for rec in data.get("allPlays")])

    data = json.load(infile)
    atbats = [match.value for match in parse('allPlays[*]').find(data)]

    columns = ['gameID', 'eventNum', 'inning', 'half', 'atbat', 'balls',
               'strikes', 'outs', 'pitches', 'homeScore', 'awayScore', 'event', 'result']
    
    for atbat in atbats:
        for play in atbat["playEvents"]:
            if not play["details"].get("event"):
                continue
            print(f'{atbat["about"]["inning"]} {play["details"]["event"]}')
        print(f'{atbat["about"]["inning"]} {atbat["about"]["halfInning"]:10s} {atbat["result"]["event"]}')


In [ ]:
atbats.groupby("event").count()

In [ ]:
import json
import pandas as pd
from jsonpath_ng import jsonpath, parse

filename = f'../data/schedule/2018-08-19.json'
with open(filename, 'r') as infile:
    data = json.load(infile)
    games = [match.value for match in parse('dates[*].games').find(data)][0]
    
    for game in games:
        print(game["gamePk"])



In [3]:
import feather, glob
import pandas as pd

files = glob.glob(f'../data/parsed/2019/*.ftr')

large = pd.DataFrame()
li = []

for filename in files:
    df = feather.read_dataframe(filename)
    li.append(df)

total = pd.concat(li)


In [23]:
blah = total[["homeTeam", "seriesDescription", "gamePk"]].groupby(["seriesDescription", "homeTeam"]).nunique("gamePk")